In [1]:
# Import pandas, numpy, matplotlib
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from pygeocoder import Geocoder
from geopy.geocoders import Nominatim
from geopy.distance import vincenty



In [5]:
# Pull the comprehensive police reported incidents file, dated from 2007 to 2013.
incidents7_13 = pd.read_csv('incidents-100k.csv')
incidents7_13.head(2)

,id,date,year,month,day,week,dow,time,hour,is_night,...,comm_pop,council,coun_pop,asr_zone,lampdist,lat,lon,desc,gctype,gcquality
0,NaN,9/19/08,2008,9,3184,37,5,13:00:00,13,0,...,42779,San005,143957,1,3162,32.957337,-117.143777,ILLEGAL POSSESS TEAR GAS/ETC,cns/segment,54
1,NaN,7/20/08,2008,7,3123,28,0,22:00:00,22,1,...,15748,San003,147116,3,8000,32.758153,-117.125022,FRAUD,cns/segment,65
